In [3]:
import os
import json
import warnings
from datetime import datetime

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, StratifiedKFold, RandomizedSearchCV, cross_val_score
from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score, classification_report, confusion_matrix
from xgboost import XGBClassifier
import shap
import joblib
from scipy.stats import randint, uniform
!pip install lime
from lime.lime_tabular import LimeTabularExplainer

warnings.filterwarnings("ignore")
np.random.seed(42)

# ===========
# Configuration
# ===========
DATA_FILE = "credit_risk.csv"   # Input dataset filename
TARGET_COL = "label"               # 0 = accepted, 1 = rejected  <-- Corrected: Changed from "id" to "label"
OUTPUT_DIR = "output"
N_JOBS = 4

# Create output directory
os.makedirs(OUTPUT_DIR, exist_ok=True)

# ===========
# Helpers
# ===========
def save_json(obj, path):
    with open(path, "w") as f:
        json.dump(obj, f, indent=2)

def save_text(text, path):
    with open(path, "w", encoding="utf8") as f:
        f.write(text)

def plot_and_save_feature_importance(features, importances, outpath):
    order = np.argsort(importances)
    plt.figure(figsize=(8, max(4, 0.4 * len(features))))
    plt.barh(np.array(features)[order], importances[order])
    plt.title("XGBoost feature importances")
    plt.xlabel("Importance")
    plt.tight_layout()
    plt.savefig(outpath)
    plt.close()

def safe_shap_summary_plot(explainer, shap_values, X, outpath):
    # shap.summary_plot creates its own matplotlib figure
    shap.summary_plot(shap_values, X, show=False)
    plt.tight_layout()
    plt.savefig(outpath, bbox_inches="tight")
    plt.close()

def safe_shap_force_plot(explainer, shap_values, X_row, outpath):
    # For a single instance produce a force plot and save as PNG using matplotlib
    # Use shap.plots._force.matplotlib_force_plot for matplotlib rendering
    try:
        fp = shap.plots._force.matplotlib_force_plot(explainer.expected_value, shap_values, X_row, show=False)
        plt.savefig(outpath, bbox_inches="tight")
        plt.close()
    except Exception as e:
        # fallback: save a small text summary
        with open(outpath.replace(".png", ".txt"), "w") as f:
            f.write("Could not render force plot: " + str(e))

# ===========
# 1. Load data
# ===========
data=pd.read_csv("/content/customer_data.csv",encoding='latin-1')
df = pd.DataFrame(data)
if TARGET_COL not in df.columns:
    raise ValueError(f"Target column '{TARGET_COL}' not found in dataset. Columns found: {list(df.columns)}")

# Basic check and feature list
feature_cols = [c for c in df.columns if c != TARGET_COL]
print("Features detected:", feature_cols)

# ===========
# 2. Preprocessing
#    - simple: numeric fill with median, categorical label encoding if any
#    - keep pipeline simple so reviewer can reproduce
# ===========
X = df[feature_cols].copy()
y = df[TARGET_COL].copy()

# Auto-detect numeric vs categorical
numeric_cols = X.select_dtypes(include=[np.number]).columns.tolist()
cat_cols = [c for c in X.columns if c not in numeric_cols]

# Fill numeric missing with median
for c in numeric_cols:
    if X[c].isnull().any():
        X[c] = X[c].fillna(X[c].median())

# For categorical columns simple factorize
for c in cat_cols:
    if X[c].isnull().any():
        X[c] = X[c].fillna("MISSING")
    X[c], _ = pd.factorize(X[c])

# Final feature list
features = X.columns.tolist()

# ===========
# 3. Train test split
# ===========
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, stratify=y, random_state=42
)
print("Train size:", X_train.shape, "Test size:", X_test.shape)

# ===========
# 4. Baseline XGBoost and hyperparameter tuning
#    - RandomizedSearchCV with small search to keep runtime reasonable
# ===========
base_model = XGBClassifier(
    n_estimators=200,
    use_label_encoder=False,
    eval_metric="logloss",
    random_state=42,
    n_jobs=N_JOBS
)

param_dist = {
    "max_depth": randint(3, 8),
    "learning_rate": uniform(0.01, 0.2),
    "subsample": uniform(0.6, 0.4),
    "colsample_bytree": uniform(0.6, 0.4),
    "n_estimators": randint(100, 400)
}

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
search = RandomizedSearchCV(
    estimator=base_model,
    param_distributions=param_dist,
    n_iter=20,
    scoring="roc_auc",
    cv=cv,
    random_state=42,
    n_jobs=N_JOBS,
    verbose=1
)

print("Starting hyperparameter search")
search.fit(X_train, y_train)
best_model = search.best_estimator_
print("Best params:", search.best_params_)
print("Best CV AUC:", search.best_score_)

# Save search results
save_json({
    "best_params": search.best_params_,
    "best_score_cv_auc": float(search.best_score_)
}, os.path.join(OUTPUT_DIR, "tuning_results.json"))

# ===========
# 5. Cross validation on full training set with best model
# ===========
cv_scores = cross_val_score(best_model, X_train, y_train, cv=cv, scoring="roc_auc", n_jobs=N_JOBS)
cv_scores_list = [float(s) for s in cv_scores]
print("CV AUC scores on train folds:", cv_scores_list)
save_json({"cv_auc_train_folds": cv_scores_list}, os.path.join(OUTPUT_DIR, "cv_train_folds.json"))

# ===========
# 6. Final fit on full training set and evaluate on test set
# ===========
best_model.fit(X_train, y_train)

# Predictions
y_proba = best_model.predict_proba(X_test)[:, 1]
y_pred = best_model.predict(X_test)

metrics = {
    "accuracy": float(accuracy_score(y_test, y_pred)),
    "roc_auc": float(roc_auc_score(y_test, y_proba)),
    "precision": float(precision_score(y_test, y_pred, zero_division=0)),
    "recall": float(recall_score(y_test, y_pred, zero_division=0))
}
print("Test metrics:", metrics)
save_json(metrics, os.path.join(OUTPUT_DIR, "test_metrics.json"))

# Confusion matrix and classification report
cm = confusion_matrix(y_test, y_pred).tolist()
cr = classification_report(y_test, y_pred, zero_division=0, output_dict=True)
save_json({"confusion_matrix": cm, "classification_report": cr}, os.path.join(OUTPUT_DIR, "test_classification.json"))

# Save trained model
joblib.dump(best_model, os.path.join(OUTPUT_DIR, "xgb_model.joblib"))

# ===========
# 7. Global feature importance and SHAP
# ===========
# XGBoost feature importances
importances = best_model.feature_importances_
plot_and_save_feature_importance(features, importances, os.path.join(OUTPUT_DIR, "xgb_feature_importance.png"))

# SHAP tree explainer
explainer = shap.TreeExplainer(best_model)
shap_values = explainer.shap_values(X_train)  # shape: (n_samples, n_features) or list for multioutput

# Save SHAP summary plot
safe_shap_summary_plot(explainer, shap_values, X_train, os.path.join(OUTPUT_DIR, "shap_summary.png"))

# Save global feature importance numeric values
global_shap_importance = pd.DataFrame({
    "feature": features,
    "xgb_importance": importances,
    "mean_abs_shap": np.abs(shap_values).mean(axis=0).tolist() if shap_values is not None and hasattr(shap_values, "shape") else []
})
global_shap_importance = global_shap_importance.sort_values("mean_abs_shap", ascending=False)
global_shap_importance.to_csv(os.path.join(OUTPUT_DIR, "global_feature_importance.csv"), index=False)

# ===========
# 8. Select 5 representative loan applications: 3 accepted, 2 rejected
#    - If not enough in test set, pick from whole dataset but prefer test set for honest evaluation
# ===========
selected_indices = []
accepted_mask = (y_test == 0)
rejected_mask = (y_test == 1)

if accepted_mask.sum() >= 3 and rejected_mask.sum() >= 2:
    accepted_idx = list(X_test[accepted_mask].index[:3])
    rejected_idx = list(X_test[rejected_mask].index[:2])
else:
    # fallback to using whole data
    accepted_idx = list(X[y == 0].index[:3])
    rejected_idx = list(X[y == 1].index[:2])

selected_indices = accepted_idx + rejected_idx
print("Selected case indices for local explanations:", selected_indices)

# ===========
# 9. Local explanations with SHAP and LIME for each selected case
# ===========
lime_explainer = LimeTabularExplainer(
    training_data=np.array(X_train),
    feature_names=features,
    class_names=["accepted", "rejected"],
    discretize_continuous=True,
    random_state=42
)

local_summaries = []
for i, idx in enumerate(selected_indices):
    row = X.loc[idx]
    row_df = row.to_frame().T
    proba = best_model.predict_proba(row_df)[:, 1][0]
    pred = int(best_model.predict(row_df)[0])
    shap_val = explainer.shap_values(row_df).reshape(-1) if hasattr(explainer.shap_values(row_df), "reshape") else explainer.shap_values(row_df)
    # SHAP local contribution table
    shap_contrib_df = pd.DataFrame({
        "feature": features,
        "shap_value": shap_val if isinstance(shap_val, (list, np.ndarray)) else np.array(shap_val).reshape(-1),
        "value": row.values
    }).sort_values("shap_value", key=lambda s: np.abs(s), ascending=False)
    shap_contrib_df.to_csv(os.path.join(OUTPUT_DIR, f"case_{i+1}_shap_contributions.csv"), index=False)

    # Save SHAP force plot (matplotlib fallback)
    out_force = os.path.join(OUTPUT_DIR, f"case_{i+1}_shap_force.png")
    try:
        # shap.plots.force for JS requires notebook, so use matplotlib fallback
        safe_shap_force_plot(explainer, explainer.shap_values(row_df), row_df, out_force)
    except Exception:
        pass

    # LIME explanation
    lime_exp = lime_explainer.explain_instance(
        data_row=row.values,
        predict_fn=best_model.predict_proba,
        num_features=min(10, len(features))
    )
    lime_list = lime_exp.as_list()
    lime_html = lime_exp.as_html()
    with open(os.path.join(OUTPUT_DIR, f"case_{i+1}_lime.html"), "w", encoding="utf8") as f:
        f.write(lime_html)

    # Save textual transcription combining SHAP and LIME
    lines = []
    lines.append(f"Case {i+1} index: {idx}")
    lines.append(f"Predicted class: {'rejected' if pred==1 else 'accepted'}")
    lines.append(f"Predicted probability of rejection: {proba:.4f}")
    lines.append("\nTop SHAP contributions (sorted by absolute value):")
    for _, r in shap_contrib_df.head(6).iterrows():
        lines.append(f" - {r['feature']}: value={r['value']}, shap_value={r['shap_value']:.4f}")
    lines.append("\nLIME local explanation top features:")
    for feat, val in lime_list:
        lines.append(f" - {feat}")
    transcription_text = "\n".join(lines)
    save_text(transcription_text, os.path.join(OUTPUT_DIR, f"case_{i+1}_transcription.txt"))

    local_summaries.append({
        "case": i+1,
        "index": int(idx),
        "pred": int(pred),
        "proba_reject": float(proba),
        "top_shap": shap_contrib_df.head(6).to_dict(orient="records"),
        "lime": lime_list
    })

# Save local summaries json
save_json(local_summaries, os.path.join(OUTPUT_DIR, "local_explanations_summary.json"))

# ===========
# 10. Comparative analysis text between SHAP and LIME for this task
# ===========
comparative_text = []
comparative_text.append("Comparative analysis: SHAP versus LIME for credit risk application")
comparative_text.append("")
comparative_text.append("SHAP summary")
comparative_text.append(" - SHAP provides consistent additive feature attributions.")
comparative_text.append(" - SHAP is useful for global and local interpretation.")
comparative_text.append(" - SHAP values are stable when using the same model and data.")
comparative_text.append("")
comparative_text.append("LIME summary")
comparative_text.append(" - LIME gives intuitive local linear explanations around a prediction.")
comparative_text.append(" - LIME can be less stable across different perturbation seeds.")
comparative_text.append(" - LIME requires careful choice of kernel and perturbation settings.")
comparative_text.append("")
comparative_text.append("Strengths and weaknesses for this credit risk task")
comparative_text.append(" - SHAP is preferred for regulatory justification because it gives a consistent global story and per instance additive contributions.")
comparative_text.append(" - LIME can be useful in operations to present a simple, short explanation to non technical users but should be used with caution because of stability issues.")
comparative_text.append(" - Using both methods together gives complementary evidence: SHAP for robust ranking and LIME for short local narrative.")
comparative_text.append("")
comparative_text.append("Suggested usage")
comparative_text.append(" - Use SHAP for model validation, feature selection, and reporting top drivers.")
comparative_text.append(" - Use LIME when a short, human friendly explanation must be shown to a loan officer, with the caveat that repeated runs can vary.")
save_text("\n".join(comparative_text), os.path.join(OUTPUT_DIR, "comparative_shap_vs_lime.txt"))

# ===========
# 11. Executive summary one page plain text
# ===========
top3 = global_shap_importance.head(3)
exec_lines = []
exec_lines.append("Executive summary")
exec_lines.append("")
exec_lines.append("Project objective")
exec_lines.append(" - Build an interpretable classifier for loan default risk with global and local explanations.")
exec_lines.append("")
exec_lines.append("Model performance on held out test set")
exec_lines.append(f" - Area under ROC curve (AUC): {metrics['roc_auc']:.4f}")
exec_lines.append(f" - Accuracy: {metrics['accuracy']:.4f}")
exec_lines.append(f" - Precision: {metrics['precision']:.4f}")
exec_lines.append(f" - Recall: {metrics['recall']:.4f}")
exec_lines.append("")
exec_lines.append("Top three global drivers of rejection risk (from SHAP mean absolute values):")
for _, row in top3.iterrows():
    exec_lines.append(f" - {row['feature']}: mean absolute SHAP = {row['mean_abs_shap']:.4f}")
exec_lines.append("")
exec_lines.append("Key recommendations")
exec_lines.append(" - Investigate and mitigate the top features driving rejection risk.")
exec_lines.append(" - Consider collecting higher quality data for the features with highest importance.")
exec_lines.append(" - For operational explanations, present LIME style summaries to loan officers supplemented by SHAP global evidence for regulatory reports.")
exec_lines.append("")
exec_lines.append("Files produced")
exec_lines.append(" - test_metrics.json: test evaluation metrics")
exec_lines.append(" - global_feature_importance.csv: global feature ranking")
exec_lines.append(" - case_*_transcription.txt: local explanation transcriptions")
exec_lines.append(" - case_*_lime.html: LIME interactive html explanations")
exec_lines.append(" - shap_summary.png: SHAP summary visualization")
save_text("\n".join(exec_lines), os.path.join(OUTPUT_DIR, "executive_summary.txt"))

# ===========
# 12. Final README of outputs mapping to rubric
# ===========
mapping_lines = []
mapping_lines.append("Deliverables mapping to the rubric")
mapping_lines.append("")
mapping_lines.append("Task 1: Model development and tuning")
mapping_lines.append(" - Provided: xgb_model.joblib and tuning_results.json and cv_train_folds.json")
mapping_lines.append("")
mapping_lines.append("Task 2: Global feature importance visualizations and interpretations")
mapping_lines.append(" - Provided: xgb_feature_importance.png and shap_summary.png and global_feature_importance.csv")
mapping_lines.append("")
mapping_lines.append("Task 3: Local explanations for 5 case studies")
mapping_lines.append(" - Provided: case_1..case_5_transcription.txt and case_1..case_5_shap_contributions.csv and case_1..case_5_lime.html")
mapping_lines.append("")
mapping_lines.append("Task 4: Comparative analysis of SHAP vs LIME")
mapping_lines.append(" - Provided: comparative_shap_vs_lime.txt")
mapping_lines.append("")
mapping_lines.append("Task 5: Executive summary text")
mapping_lines.append(" - Provided: executive_summary.txt")
save_text("\n".join(mapping_lines), os.path.join(OUTPUT_DIR, "deliverables_mapping.txt"))

print("All outputs saved to", OUTPUT_DIR)
print("Contents:", os.listdir(OUTPUT_DIR))
print("Script finished at", datetime.now().isoformat())

Features detected: ['id', 'fea_1', 'fea_2', 'fea_3', 'fea_4', 'fea_5', 'fea_6', 'fea_7', 'fea_8', 'fea_9', 'fea_10', 'fea_11']
Train size: (843, 12) Test size: (282, 12)
Starting hyperparameter search
Fitting 5 folds for each of 20 candidates, totalling 100 fits
Best params: {'colsample_bytree': np.float64(0.7098887171960256), 'learning_rate': np.float64(0.12224868516954022), 'max_depth': 5, 'n_estimators': 200, 'subsample': np.float64(0.9886848381556415)}
Best CV AUC: 0.6213972226410036
CV AUC scores on train folds: [0.6400871459694989, 0.6553376906318082, 0.6056644880174292, 0.6507901668129938, 0.5551066217732884]
Test metrics: {'accuracy': 0.7659574468085106, 'roc_auc': 0.5099557522123894, 'precision': 0.2222222222222222, 'recall': 0.07142857142857142}
Selected case indices for local explanations: [30, 482, 352, 142, 276]
All outputs saved to output
Contents: ['xgb_feature_importance.png', 'case_5_shap_force.txt', 'case_2_shap_contributions.csv', 'case_1_shap_contributions.csv', 'ca